<a href="https://colab.research.google.com/github/brayanlolv/geranadoquestoes/blob/main/trabalho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##instalando variaveis

In [ ]:
!pip install lmqg

# *##SETANDO O LMQG E SUAS FUNCOES\*





In [ ]:
from lmqg import TransformersQG
model = TransformersQG(language = "en")


In [ ]:
import numpy as np
import math


def createquestion(context,language = "en"):
  ## tratar o tamanho do context max 512

  #dividir o context
  print("create question chamada")
  contextsplit = math.floor((len(context) / 512) +1)
  print(contextsplit)
  slicesize = math.floor(len(context) / contextsplit)
  print(slicesize)

  result = []


  for i in range(contextsplit+1):

    if(i>3):
      break ##quebrado

    print("for")
    a = slicesize * i
    b = slicesize * (i + 1)
    print(i)
    print(a)
    print(b)

    try:
      question = model.generate_qa(context[a:b])
      result.extend(question)
    except:
      continue

    print("criando questoes")

  return result# tentando slice




##TB get text


In [ ]:
!pip install youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse


#adiconar lingua
def get_videoscriptbyurl(video_url,language = "en"):
  video_id = urlparse(video_url)
  video_id = video_id.query.replace("v=","")

  transcript_array =  []
  transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])

  for x in transcript:
    # print(x["text"])
    transcript_array.append(x["text"])

  return transcript_array

print(

)



##iniciando o flask


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!zip -r /content/gdrive/MyDrive/templates.zip  . -i /content/gdrive/Mydrive/templates/
# !zip -r /content/olala.zip . /content/gdrive/Mydrive/templates

In [ ]:
from flask import Flask, redirect, jsonify,  render_template,request
serve = Flask("ola", template_folder="/content/gdrive/MyDrive/templates")

from flask import make_response



@serve.route("/")
def home():
  # return "ola"
  alert = ""
  return render_template('index.html',alert=alert)



#endpoint para gerar as questoes a partir de um video
@serve.route("/yb_post", methods = ["POST"])
def youtube():
  data = {}
  data["video_url"] = request.form.get('video_url')



  data["text-context"] = get_videoscriptbyurl(data["video_url"])
  data["questions"] = createquestion(" ".join(data["text-context"]))

  #so para fazer o front
  #data["questions"] = [('In what year did Hitler leave a disturbingmark on the automobile industry?', '1937 he left a very disturbingmark on the automobile industry')]
  return render_template("ybpost.html",data=data)

#endpoit para gerar as questoes a partir de um text
@serve.route("/text_post", methods = ["POST"])
def questionText():
  print("text post")
  data = {}
  data["text-context"] = request.form.get('text_context')
  data["questions"] = createquestion("".join(data["text-context"]))

  return render_template('textpost.html',data=data)


from google.colab import output
output.serve_kernel_port_as_window(5000)


serve.run()